In [91]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import datetime as dt
import requests
import json
import os
from bs4 import BeautifulSoup
import time
import random
import concurrent.futures
from typing import List, Tuple, Dict, Any
import logging
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from threading import Lock
import gc

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure session with retries and connection pooling
session = requests.Session()
retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries, pool_connections=100, pool_maxsize=100))
session.mount('https://', HTTPAdapter(max_retries=retries, pool_connections=100, pool_maxsize=100))

# Thread-safe lock for file writing
file_lock = Lock()

In [92]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [93]:
def calculate_rsi(data, window=14):
    # Ensure the 'Close' column is correctly accessed
    if isinstance(data.columns, pd.MultiIndex):
        close_prices = data['Close'].iloc[:, 0]
    else:
        close_prices = data['Close']

    # Calculate price changes
    delta = close_prices.diff()

    # Separate gains (positive) and losses (negative)
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)

    # Initialize the averages
    avg_gains = [np.nan] * len(close_prices)
    avg_losses = [np.nan] * len(close_prices)

    # Calculate first averages after initial window
    first_avg_gain = gains[1:window+1].mean()
    first_avg_loss = losses[1:window+1].mean()
    avg_gains[window] = first_avg_gain
    avg_losses[window] = first_avg_loss

    # Calculate subsequent values using the Wilder's smoothing method
    for i in range(window+1, len(close_prices)):
        avg_gain = (avg_gains[i-1] * (window-1) + gains[i]) / window
        avg_loss = (avg_losses[i-1] * (window-1) + losses[i]) / window
        avg_gains[i] = avg_gain
        avg_losses[i] = avg_loss

    # Convert to Series with proper index
    avg_gains = pd.Series(avg_gains, index=close_prices.index)
    avg_losses = pd.Series(avg_losses, index=close_prices.index)

    # Calculate RS and RSI
    rs = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [94]:
def get_stock_data(symbol, period='1y'):
    """Fetch stock data with proper error handling and ensure unique data per stock"""
    try:
        time.sleep(1)  # Add delay to prevent rate limiting

        # Create a new ticker instance for each request
        ticker = yf.Ticker(f"{symbol}.NS")

        # Force a fresh download with unique session
        data = yf.download(
            tickers=f"{symbol}.NS",
            period=period,
            progress=False,
            threads=False,  # Disable multi-threading to prevent data mixing
            ignore_tz=True,
            auto_adjust=True,  # Adjust OHLC automatically
            prepost=False,  # Use only regular trading hours
            repair=True  # Repair and validate the data
        )

        if data.empty:
            logger.warning(f"No data available for {symbol}")
            return pd.DataFrame()

        # Create a fresh DataFrame for this stock's data
        df = pd.DataFrame(index=data.index)

        # Copy each column individually
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            if col in data.columns:
                df[col] = data[col].copy()
            else:
                logger.warning(f"Column {col} not found for {symbol}")
                df[col] = np.nan

        # Verify data uniqueness
        if 'Close' in df.columns:
            latest_price = df['Close'].iloc[-1]
            logger.info(f"Verified unique data for {symbol}: Latest price = ₹{latest_price:.2f}")

        return df

    except Exception as e:
        logger.error(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()

In [95]:
def calculate_macd(data, fast=12, slow=26, signal=9):
    try:
        # Make sure we're working with a copy of the data to avoid warnings
        if isinstance(data.columns, pd.MultiIndex):
            close_series = data['Close'].iloc[:, 0].copy()
        else:
            close_series = data['Close'].copy()

        # Calculate EMAs
        ema_fast = close_series.ewm(span=fast, adjust=False).mean()
        ema_slow = close_series.ewm(span=slow, adjust=False).mean()

        # Calculate MACD components
        macd_line = ema_fast - ema_slow
        signal_line = macd_line.ewm(span=signal, adjust=False).mean()
        histogram = macd_line - signal_line

        return macd_line, signal_line, histogram
    except Exception as e:
        logger.error(f"Error calculating MACD: {e}")
        empty_series = pd.Series(dtype=float)
        return empty_series, empty_series, empty_series

In [96]:
def calculate_momentum(data, period=20):
    try:
        if isinstance(data.columns, pd.MultiIndex):
            close_series = data['Close'].iloc[:, 0]
        else:
            close_series = data['Close']

        momentum = close_series / close_series.shift(period) - 1
        return momentum * 100
    except Exception as e:
        logger.error(f"Error calculating momentum: {e}")
        return pd.Series(dtype=float)

In [97]:
def calculate_momentum_index(data, period=126):
    try:
        if isinstance(data.columns, pd.MultiIndex):
            close_series = data['Close'].iloc[:, 0]
        else:
            close_series = data['Close']

        returns = close_series.pct_change().dropna()
        momentum_std = returns.rolling(window=period).std() * np.sqrt(252)
        return momentum_std
    except Exception as e:
        logger.error(f"Error calculating momentum index: {e}")
        return pd.Series(dtype=float)

In [98]:
def get_fundamental_data(symbol: str) -> Dict[str, Any]:
    """Fetch fundamental data"""
    try:
        ticker = yf.Ticker(f"{symbol}.NS")
        info = ticker.info
        fundamental_data = {
            'P/E Ratio': info.get('trailingPE', 'N/A'),
            'Forward P/E': info.get('forwardPE', 'N/A'),
            'Market Cap': info.get('marketCap', 'N/A'),
            'EPS': info.get('trailingEps', 'N/A'),
            'Dividend Yield': info.get('dividendYield', 'N/A'),
            'Debt to Equity': info.get('debtToEquity', 'N/A'),
            'Return on Equity': info.get('returnOnEquity', 'N/A'),
            'Revenue Growth': info.get('revenueGrowth', 'N/A'),
            'Profit Margins': info.get('profitMargins', 'N/A'),
            'Beta': info.get('beta', 'N/A'),
            'Current Ratio': info.get('currentRatio', 'N/A'),
            'Book Value': info.get('bookValue', 'N/A'),
            '52-Week High': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52-Week Low': info.get('fiftyTwoWeekLow', 'N/A'),
            'Target Price': info.get('targetMeanPrice', 'N/A')
        }
        return fundamental_data
    except Exception as e:
        logger.error(f"Error fetching fundamental data for {symbol}: {e}")
        return {}

In [99]:
def determine_strength(data, rsi, macd_line, signal_line):
    # Ensure data is not empty
    if isinstance(data, pd.DataFrame) and data.empty:
        return [], []

    # Handle multi-index columns if they exist
    if isinstance(data.columns, pd.MultiIndex):
        # Extract the first level columns if multi-index
        data_cols = {
            'Open': data['Open'].iloc[:, 0],
            'High': data['High'].iloc[:, 0],
            'Low': data['Low'].iloc[:, 0],
            'Close': data['Close'].iloc[:, 0],
            'Volume': data['Volume'].iloc[:, 0]
        }
        data_df = pd.DataFrame(data_cols, index=data.index)
    else:
        data_df = data

    current_price = data_df['Close'].iloc[-1]
    sma_50 = data_df['Close'].rolling(window=50).mean().iloc[-1]
    sma_200 = data_df['Close'].rolling(window=200).mean().iloc[-1]

    # Convert any Series to scalar values
    if isinstance(current_price, pd.Series):
        current_price = current_price.iloc[0]
    if isinstance(sma_50, pd.Series):
        sma_50 = sma_50.iloc[0]
    if isinstance(sma_200, pd.Series):
        sma_200 = sma_200.iloc[0]

    strengths = []
    weaknesses = []

    # RSI Analysis
    if not isinstance(rsi, pd.Series) or len(rsi) == 0:
        print("RSI data is empty")
    else:
        last_rsi = rsi.iloc[-1]
        if isinstance(last_rsi, pd.Series):
            last_rsi = last_rsi.item()
        if last_rsi > 70:
            weaknesses.append("RSI indicates overbought conditions")
        elif last_rsi < 30:
            strengths.append("RSI indicates oversold conditions (potential buying opportunity)")
        elif 40 <= last_rsi <= 60:
            strengths.append("RSI in neutral zone showing balance between buyers and sellers")
        elif 60 < last_rsi < 70:
            strengths.append("Strong RSI showing positive momentum")

    # MACD Analysis
    if (not isinstance(macd_line, pd.Series) or len(macd_line) == 0 or
        not isinstance(signal_line, pd.Series) or len(signal_line) == 0):
        print("MACD or signal line data is empty")
    else:
        macd_value = macd_line.iloc[-1]
        signal_value = signal_line.iloc[-1]
        if isinstance(macd_value, pd.Series):
            macd_value = macd_value.item()
        if isinstance(signal_value, pd.Series):
            signal_value = signal_value.item()
        if macd_value > signal_value:
            strengths.append("MACD line above signal line indicating bullish momentum")
        else:
            weaknesses.append("MACD line below signal line indicating bearish momentum")

    # Moving Average Analysis
    if pd.notna(sma_50) and pd.notna(current_price):
        if current_price > sma_50:
            strengths.append("Price above 50-day SMA showing short-term strength")
        else:
            weaknesses.append("Price below 50-day SMA showing short-term weakness")

    if pd.notna(sma_200) and pd.notna(current_price):
        if current_price > sma_200:
            strengths.append("Price above 200-day SMA suggesting long-term uptrend")
        else:
            weaknesses.append("Price below 200-day SMA suggesting long-term downtrend")

    # Golden/Death Cross
    if pd.notna(sma_50) and pd.notna(sma_200):
        if sma_50 > sma_200 and (sma_50 / sma_200 - 1) < 0.03:
            strengths.append("Recent golden cross or nearing golden cross (50-day SMA crossing above 200-day SMA)")
        elif sma_50 < sma_200 and (sma_200 / sma_50 - 1) < 0.03:
            weaknesses.append("Recent death cross or nearing death cross (50-day SMA crossing below 200-day SMA)")

    # Volume Analysis
    avg_volume = data_df['Volume'].mean()
    if isinstance(avg_volume, pd.Series):
        avg_volume = avg_volume.item()

    recent_volume = data_df['Volume'].iloc[-5:].mean()
    if isinstance(recent_volume, pd.Series):
        recent_volume = recent_volume.item()

    # Calculate price trend
    recent_price_mean = data_df['Close'].iloc[-5:].mean()
    if isinstance(recent_price_mean, pd.Series):
        recent_price_mean = recent_price_mean.item()

    price_trend_up = current_price > recent_price_mean

    if recent_volume > avg_volume * 1.2:
        if price_trend_up:
            strengths.append("Strong volume supporting upward price movement")
        else:
            weaknesses.append("High volume during price decline indicates selling pressure")
    elif recent_volume < avg_volume * 0.8:
        if price_trend_up:
            strengths.append("Price rising on low volume - potential weakness")
        else:
            weaknesses.append("Price declining on low volume - potential for reversal")

    # Price Movement
    if len(data_df['Close']) >= 22:
        latest_price = data_df['Close'].iloc[-1]
        price_22_days_ago = data_df['Close'].iloc[-22]

        if isinstance(latest_price, pd.Series):
            latest_price = latest_price.item()
        if isinstance(price_22_days_ago, pd.Series):
            price_22_days_ago = price_22_days_ago.item()

        monthly_return = (latest_price / price_22_days_ago - 1) * 100

        if monthly_return > 5:
            strengths.append(f"Strong monthly return of {monthly_return:.2f}%")
        elif monthly_return < -5:
            weaknesses.append(f"Weak monthly return of {monthly_return:.2f}%")

    # Volatility
    if len(data_df['Close']) >= 2:
        returns = data_df['Close'].pct_change().dropna()
        if len(returns) > 0:
            volatility = returns.std() * np.sqrt(252) * 100
            if isinstance(volatility, pd.Series):
                volatility = volatility.item()

            if volatility > 30:
                weaknesses.append(f"High volatility ({volatility:.2f}%) indicating increased risk")
            elif volatility < 15:
                strengths.append(f"Low volatility ({volatility:.2f}%) indicating stability")

    return strengths, weaknesses

In [100]:
def process_single_stock(company_name: str, symbol: str, output_dir: str) -> Dict[str, Any]:
    """Process a single stock with all calculations and verify data uniqueness"""
    result = {
        'company_name': company_name,
        'symbol': symbol,
        'success': False,
        'data': pd.DataFrame(),
        'sentiment': {},
        'markdown': ""
    }

    try:
        logger.info(f"Processing: {company_name} ({symbol})")

        # Get stock data with retries
        max_retries = 3
        for attempt in range(max_retries):
            time.sleep(2)  # Delay between attempts
            data = get_stock_data(symbol)
            if not data.empty:
                break
            logger.warning(f"Attempt {attempt + 1} failed for {symbol}, retrying...")

        if data.empty:
            logger.error(f"Failed to get data for {symbol} after {max_retries} attempts")
            return result

        # Verify we have unique data for this stock
        if 'Close' in data.columns:
            current_price = float(data['Close'].iloc[-1])
            logger.info(f"Verified price data for {symbol}: Current price = ₹{current_price:.2f}")
        else:
            logger.error(f"Invalid data format for {symbol}")
            return result

        # Calculate technical indicators
        time.sleep(1)  # Add delay between calculations
        rsi = calculate_rsi(data)
        macd_line, signal_line, histogram = calculate_macd(data)
        mom_6m = calculate_momentum_index(data, period=126)
        mom_1y = calculate_momentum_index(data, period=252)

        # Get fundamental data with retry
        time.sleep(2)  # Add delay before fundamental data fetch
        fundamentals = get_fundamental_data(symbol)

        if not fundamentals:
            logger.warning(f"No fundamental data available for {symbol}")
            fundamentals = {}  # Use empty dict to continue with technical analysis

        # Generate analysis
        strengths, weaknesses = determine_strength(data, rsi, macd_line, signal_line)
        profit_projection = create_profit_projection(data, fundamentals)
        sentiment = generate_sentiment(data, rsi, macd_line, signal_line, fundamentals, mom_6m, mom_1y)

        # Generate markdown
        stock_md = generate_stock_markdown(
            company_name, symbol, data, fundamentals, strengths, weaknesses,
            profit_projection, mom_6m, mom_1y, sentiment
        )

        # Save with atomic write
        output_file = os.path.join(output_dir, f"{symbol}.md")
        temp_file = f"{output_file}.tmp"

        try:
            with open(temp_file, "w", encoding='utf-8') as f:
                f.write(stock_md)
                f.flush()
                os.fsync(f.fileno())

            if os.path.exists(output_file):
                os.remove(output_file)
            os.rename(temp_file, output_file)

            result.update({
                'success': True,
                'data': data,
                'sentiment': sentiment,
                'markdown': stock_md
            })

            logger.info(f"Successfully processed {symbol}")
            return result

        except Exception as e:
            logger.error(f"Error saving data for {symbol}: {e}")
            if os.path.exists(temp_file):
                try:
                    os.remove(temp_file)
                except:
                    pass
            return result

    except Exception as e:
        logger.error(f"Error processing {symbol}: {e}")
        return result

In [101]:
def create_profit_projection(data, fundamentals):
    try:
        # Get historical returns
        returns = data['Close'].pct_change().dropna()
        annual_return = returns.mean() * 252 * 100

        # Ensure scalar value for annual return
        if isinstance(annual_return, pd.Series):
            annual_return = annual_return.item()

        # Calculate volatility
        volatility = returns.std() * np.sqrt(252) * 100
        if isinstance(volatility, pd.Series):
            volatility = volatility.item()

        # Get current price as scalar
        current_price = data['Close'].iloc[-1]
        if isinstance(current_price, pd.Series):
            current_price = current_price.item()
        current_price = float(current_price)

        # Calculate Sharpe ratio (assuming 2.5% risk-free rate)
        sharpe_ratio = (annual_return - 2.5) / volatility if volatility > 0 else 0
        risk_adjustment = 1 + (sharpe_ratio / 10)

        # Calculate growth scenarios with risk adjustment
        conservative_growth = max(annual_return * 0.5 * risk_adjustment, 2)
        moderate_growth = max(annual_return * risk_adjustment, 5)
        aggressive_growth = max(annual_return * 1.5 * risk_adjustment, 8)

        # Calculate price targets
        conservative_target = current_price * (1 + conservative_growth/100)
        moderate_target = current_price * (1 + moderate_growth/100)
        aggressive_target = current_price * (1 + aggressive_growth/100)

        # Adjust projections based on fundamentals if available
        pe_ratio = fundamentals.get('P/E Ratio', None)
        if pe_ratio and pe_ratio != 'N/A':
            pe_ratio = float(pe_ratio)
            if pe_ratio > 25:  # Higher P/E suggests overvaluation
                conservative_target *= 0.9
                moderate_target *= 0.95
            elif pe_ratio < 15:  # Lower P/E suggests undervaluation
                moderate_target *= 1.05
                aggressive_target *= 1.1

        return {
            'Current Price': round(current_price, 2),
            'Annual Return (Historical)': f"{annual_return:.2f}%",
            'Volatility': f"{volatility:.2f}%",
            'Sharpe Ratio': f"{sharpe_ratio:.2f}",
            'Conservative 1-Year Target': round(conservative_target, 2),
            'Moderate 1-Year Target': round(moderate_target, 2),
            'Aggressive 1-Year Target': round(aggressive_target, 2)
        }
    except Exception as e:
        print(f"Error creating profit projection: {e}")
        return {
            'Current Price': float(data['Close'].iloc[-1]) if not data.empty else 'N/A',
            'Annual Return (Historical)': 'N/A',
            'Volatility': 'N/A',
            'Sharpe Ratio': 'N/A',
            'Conservative 1-Year Target': 'N/A',
            'Moderate 1-Year Target': 'N/A',
            'Aggressive 1-Year Target': 'N/A'
        }

In [102]:
def generate_sentiment(data, rsi, macd_line, signal_line, fundamentals, mom_6m, mom_1y):
    """Generate sentiment analysis with optimized data handling"""
    try:
        # Handle multi-index columns if they exist
        if isinstance(data.columns, pd.MultiIndex):
            data_df = pd.DataFrame({
                'Close': data['Close'].iloc[:, 0],
                'Volume': data['Volume'].iloc[:, 0]
            })
        else:
            data_df = data[['Close', 'Volume']]

        # Extract scalar values safely
        rsi_value = float(rsi.iloc[-1].item() if isinstance(rsi.iloc[-1], pd.Series) else rsi.iloc[-1])
        macd_value = float(macd_line.iloc[-1].item() if isinstance(macd_line.iloc[-1], pd.Series) else macd_line.iloc[-1])
        signal_value = float(signal_line.iloc[-1].item() if isinstance(signal_line.iloc[-1], pd.Series) else signal_line.iloc[-1])

        # Technical signals with vectorized operations where possible
        rsi_signal = 1 if 40 < rsi_value < 70 else -1
        macd_signal = 1 if macd_value > signal_value else -1

        # Price trends using vectorized operations
        close_value = float(data_df['Close'].iloc[-1])
        ma_50 = float(data_df['Close'].rolling(window=50).mean().iloc[-1])
        ma_200 = float(data_df['Close'].rolling(window=200).mean().iloc[-1])

        price_above_50ma = 1 if close_value > ma_50 else -1
        price_above_200ma = 1 if close_value > ma_200 else -1

        # Momentum analysis
        mom_value = float(mom_6m.iloc[-1].item() if isinstance(mom_6m.iloc[-1], pd.Series) else mom_6m.iloc[-1])
        momentum_signal = 1 if mom_value < 0.15 else (0 if mom_value < 0.25 else -1)

        # Volume analysis using vectorized operations
        recent_vol = data_df['Volume'].iloc[-10:].mean()
        older_vol = data_df['Volume'].iloc[-30:-10].mean()
        volume_trend = 1 if recent_vol > older_vol else -1

        # Fundamental analysis
        pe_signal = 0
        if fundamentals.get('P/E Ratio', 'N/A') != 'N/A':
            try:
                pe = float(fundamentals['P/E Ratio'])
                pe_signal = 1 if pe < 20 else (0 if pe < 30 else -1)
            except (ValueError, TypeError):
                pass

        # Combine signals with weighted importance
        signals = {
            'Technical': (rsi_signal * 0.15 + macd_signal * 0.15 +
                        price_above_50ma * 0.2 + price_above_200ma * 0.3),
            'Momentum': momentum_signal * 0.1,
            'Volume': volume_trend * 0.1
        }

        if pe_signal != 0:
            signals['Fundamental'] = pe_signal * 0.2

        score = sum(signals.values()) / len(signals)

        # Determine sentiment and recommendation
        if score > 0.5:
            sentiment = "Strongly Positive"
            recommendation = "Strong Buy"
        elif score > 0:
            sentiment = "Positive"
            recommendation = "Buy"
        elif score > -0.5:
            sentiment = "Negative"
            recommendation = "Sell"
        else:
            sentiment = "Strongly Negative"
            recommendation = "Strong Sell"

        return {
            'Sentiment': sentiment,
            'Recommendation': recommendation,
            'Score': f"{score:.2f}",
            'RSI Signal': "Positive" if rsi_signal > 0 else "Negative",
            'MACD Signal': "Positive" if macd_signal > 0 else "Negative",
            'Price Trend': "Positive" if price_above_50ma > 0 else "Negative",
            'Long-term Trend': "Positive" if price_above_200ma > 0 else "Negative",
            'Momentum Signal': "Positive" if momentum_signal > 0 else ("Neutral" if momentum_signal == 0 else "Negative"),
            'Volume Trend': "Positive" if volume_trend > 0 else "Negative"
        }
    except Exception as e:
        logger.error(f"Error generating sentiment: {e}")
        return {
            'Sentiment': "Neutral",
            'Recommendation': "Hold",
            'Score': "0.00",
            'RSI Signal': "N/A",
            'MACD Signal': "N/A",
            'Price Trend': "N/A",
            'Long-term Trend': "N/A",
            'Momentum Signal': "N/A",
            'Volume Trend': "N/A"
        }

# Parse the stock list from the user input
def parse_stock_list(stock_list_text):
    stocks = []
    lines = stock_list_text.strip().split('\n')

    for line in lines:
        if '\t' in line:
            parts = line.split('\t')
            if len(parts) >= 2:
                company_name = parts[0].strip()
                symbol = parts[1].strip()
                if company_name not in ['Company Name', ''] and symbol not in ['Symbol', '']:
                    stocks.append((company_name, symbol))

    return stocks

In [103]:
def generate_stock_markdown(company_name: str, symbol: str, data: pd.DataFrame,
                        fundamentals: Dict[str, Any], strengths: List[str],
                        weaknesses: List[str], profit_projection: Dict[str, Any],
                        mom_6m: pd.Series, mom_1y: pd.Series, sentiment: Dict[str, str]) -> str:
    """Generate markdown report with optimized data handling"""
    if data.empty:
        return f"# {company_name} ({symbol})\n\nError: Unable to fetch data for this stock."

    try:
        # Handle multi-index columns if they exist
        if isinstance(data.columns, pd.MultiIndex):
            data_df = pd.DataFrame({
                'Open': data['Open'].iloc[:, 0],
                'High': data['High'].iloc[:, 0],
                'Low': data['Low'].iloc[:, 0],
                'Close': data['Close'].iloc[:, 0],
                'Volume': data['Volume'].iloc[:, 0]
            }, index=data.index)
        else:
            data_df = data

        # Format dates for readability
        data_df.index = pd.to_datetime(data_df.index)

        # Start building markdown content
        md_content = [
            f"# {company_name} ({symbol})\n",
            f"*Last Updated: {dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n",
            "\n## Current Price and Performance\n"
        ]

        # Current price and summary (with safe type conversion)
        current_price = float(data_df['Close'].iloc[-1])
        price_change = (current_price / float(data_df['Close'].iloc[0]) - 1) * 100

        md_content.extend([
            f"**Current Price:** ₹{current_price:.2f}\n",
            f"**1-Year Change:** {price_change:.2f}%\n",
            "\n## Sentiment Analysis\n",
            f"**Sentiment:** {sentiment['Sentiment']}\n",
            f"**Recommendation:** {sentiment['Recommendation']}\n",
            f"**Sentiment Score:** {sentiment['Score']}\n",
            "\n### Sentiment Factors\n",
            f"- **RSI Signal:** {sentiment['RSI Signal']}\n",
            f"- **MACD Signal:** {sentiment['MACD Signal']}\n",
            f"- **Price Trend (50-day MA):** {sentiment['Price Trend']}\n",
            f"- **Long-term Trend (200-day MA):** {sentiment['Long-term Trend']}\n",
            f"- **Momentum Signal:** {sentiment['Momentum Signal']}\n",
            f"- **Volume Trend:** {sentiment['Volume Trend']}\n"
        ])

        # Add strengths and weaknesses
        md_content.extend(["\n## Strengths\n"] + [f"- {s}\n" for s in strengths])
        md_content.extend(["\n## Weaknesses\n"] + [f"- {w}\n" for w in weaknesses])

        # Add momentum analysis
        md_content.append("\n## Momentum Analysis\n")
        if not mom_6m.empty and not pd.isna(mom_6m.iloc[-1]):
            md_content.append(f"**6-Month Momentum Index:** {float(mom_6m.iloc[-1]):.4f}\n")
        else:
            md_content.append("**6-Month Momentum Index:** N/A\n")

        if not mom_1y.empty and not pd.isna(mom_1y.iloc[-1]):
            md_content.append(f"**1-Year Momentum Index:** {float(mom_1y.iloc[-1]):.4f}\n")
        else:
            md_content.append("**1-Year Momentum Index:** N/A\n")

        # Add technical indicators
        rsi = calculate_rsi(data_df)
        macd_line, signal_line, histogram = calculate_macd(data_df)
        sma_50 = data_df['Close'].rolling(window=50).mean().iloc[-1]
        sma_200 = data_df['Close'].rolling(window=200).mean().iloc[-1]

        md_content.extend([
            "\n## Technical Indicators\n",
            f"**RSI (14-day):** {float(rsi.iloc[-1]):.2f}\n",
            f"**MACD Line:** {float(macd_line.iloc[-1]):.2f}\n",
            f"**MACD Signal Line:** {float(signal_line.iloc[-1]):.2f}\n",
            f"**MACD Histogram:** {float(histogram.iloc[-1]):.2f}\n",
            f"**50-day SMA:** ₹{float(sma_50):.2f}\n",
            f"**200-day SMA:** ₹{float(sma_200):.2f}\n"
        ])

        # Add fundamental analysis
        md_content.append("\n## Fundamental Analysis\n")
        for key, value in fundamentals.items():
            if key in ['Dividend Yield', 'Revenue Growth', 'Profit Margins', 'Return on Equity'] and value != 'N/A':
                try:
                    value = f"{float(value) * 100:.2f}%"
                except (ValueError, TypeError):
                    pass
            md_content.append(f"**{key}:** {value}\n")

        # Add profit projections
        md_content.append("\n## Profit Projections\n")
        for key, value in profit_projection.items():
            md_content.append(f"**{key}:** {value}\n")

        # Add price history table with optimized monthly data handling
        md_content.append("\n## Price History (Monthly)\n")

        # Create monthly data using efficient resampling
        monthly_data = data_df.resample('M').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        }).sort_index(ascending=False)

        # Add table headers
        md_content.extend([
            "| Date | Open | High | Low | Close | Volume |\n",
            "|------|------|------|-----|-------|--------|\n"
        ])

        # Add table rows with efficient string formatting
        for date, row in monthly_data.iterrows():
            date_str = date.strftime('%Y-%m-%d')
            md_content.append(
                f"| {date_str} | ₹{float(row['Open']):.2f} | ₹{float(row['High']):.2f} | "
                f"₹{float(row['Low']):.2f} | ₹{float(row['Close']):.2f} | {int(row['Volume']):,} |\n"
            )

        # Join all content and return
        return "".join(md_content)

    except Exception as e:
        logger.error(f"Error generating markdown for {symbol}: {e}")
        return f"# {company_name} ({symbol})\n\nError: Unable to generate complete analysis."

In [104]:
def analyze_stocks(stock_list: List[Tuple[str, str]], output_dir: str = "stock_analysis/reports_v2", max_workers: int = 5) -> None:
    """Analyze multiple stocks in parallel with proper thread management"""
    os.makedirs(output_dir, exist_ok=True)
    total_stocks = len(stock_list)
    processed_stocks = set()  # Keep track of processed stocks

    try:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Process stocks in smaller batches to prevent resource exhaustion
            batch_size = 5
            for i in range(0, len(stock_list), batch_size):
                batch = [(company, symbol) for company, symbol in stock_list[i:i + batch_size]
                        if symbol not in processed_stocks]

                if not batch:  # Skip if batch is empty
                    continue

                # Submit batch of stocks for processing
                future_to_stock = {
                    executor.submit(process_single_stock, company_name, symbol, output_dir): (company_name, symbol)
                    for company_name, symbol in batch
                }

                # Process completed tasks
                completed_count = 0
                for future in concurrent.futures.as_completed(future_to_stock):
                    stock = future_to_stock[future]
                    symbol = stock[1]
                    completed_count += 1

                    try:
                        result = future.result()
                        status = "Success" if result['success'] else "Failed"
                        logger.info(f"Processed {completed_count}/{total_stocks}: {symbol} - {status}")
                        processed_stocks.add(symbol)  # Mark as processed
                    except Exception as e:
                        logger.error(f"Error processing {symbol}: {e}")

        logger.info(f"Analysis complete! Files saved to {output_dir}")

    except Exception as e:
        logger.error(f"Error in analyze_stocks: {e}")

In [105]:
# stock_list_text  = """Company Name	Symbol
# Rail Vikas Nigam Ltd.	RVNL
# Adani Enterprises Ltd.	ADANIENT
# Adani Ports and Special Economic Zone Ltd.	ADANIPORTS"""
# stocks_df = parse_stock_list(stock_list_text)
# analyze_stocks(stocks_df)

In [106]:
stock_list_text1 = """Company Name	Symbol
Rail Vikas Nigam Ltd.	RVNL
Adani Enterprises Ltd.	ADANIENT
Adani Ports and Special Economic Zone Ltd.	ADANIPORTS
Apollo Hospitals Enterprise Ltd.	APOLLOHOSP
Asian Paints Ltd.	ASIANPAINT
Axis Bank Ltd.	AXISBANK
Bajaj Auto Ltd.	BAJAJ-AUTO
Bajaj Finance Ltd.	BAJFINANCE
Bajaj Finserv Ltd.	BAJAJFINSV
Bharat Electronics Ltd.	BEL
Bharti Airtel Ltd.	BHARTIARTL
Cipla Ltd.	CIPLA
Coal India Ltd.	COALINDIA
Dr. Reddy's Laboratories Ltd.	DRREDDY
Eicher Motors Ltd.	EICHERMOT
Grasim Industries Ltd.	GRASIM
HCL Technologies Ltd.	HCLTECH
HDFC Bank Ltd.	HDFCBANK
HDFC Life Insurance Company Ltd.	HDFCLIFE
Hero MotoCorp Ltd.	HEROMOTOCO
Hindalco Industries Ltd.	HINDALCO
Hindustan Unilever Ltd.	HINDUNILVR
ICICI Bank Ltd.	ICICIBANK
ITC Ltd.	ITC
IndusInd Bank Ltd.	INDUSINDBK
Infosys Ltd.	INFY
JSW Steel Ltd.	JSWSTEEL
Jio Financial Services Ltd.	JIOFIN
Kotak Mahindra Bank Ltd.	KOTAKBANK
Larsen & Toubro Ltd.	LT
Mahindra & Mahindra Ltd.	M&M
Maruti Suzuki India Ltd.	MARUTI
NTPC Ltd.	NTPC
Nestle India Ltd.	NESTLEIND
Oil & Natural Gas Corporation Ltd.	ONGC
Power Grid Corporation of India Ltd.	POWERGRID
Reliance Industries Ltd.	RELIANCE
SBI Life Insurance Company Ltd.	SBILIFE
Shriram Finance Ltd.	SHRIRAMFIN
State Bank of India	SBIN
Sun Pharmaceutical Industries Ltd.	SUNPHARMA
Tata Consultancy Services Ltd.	TCS
Tata Consumer Products Ltd.	TATACONSUM
Tata Motors Ltd.	TATAMOTORS
Tata Steel Ltd.	TATASTEEL
Tech Mahindra Ltd.	TECHM
Titan Company Ltd.	TITAN
Trent Ltd.	TRENT
UltraTech Cement Ltd.	ULTRACEMCO
Wipro Ltd.	WIPRO
Zomato Ltd.	ZOMATO360 ONE WAM Ltd.	360ONE
3M India Ltd.	3MINDIA
ACC Ltd.	ACC
AIA Engineering Ltd.	AIAENG
APL Apollo Tubes Ltd.	APLAPOLLO
AU Small Finance Bank Ltd.	AUBANK
Abbott India Ltd.	ABBOTINDIA
Adani Total Gas Ltd.	ATGL
Adani Wilmar Ltd.	AWL
Aditya Birla Capital Ltd.	ABCAPITAL
Aditya Birla Fashion and Retail Ltd.	ABFRL
Ajanta Pharmaceuticals Ltd.	AJANTPHARM
Alkem Laboratories Ltd.	ALKEM
Apar Industries Ltd.	APARINDS
Apollo Tyres Ltd.	APOLLOTYRE
Ashok Leyland Ltd.	ASHOKLEY
Astral Ltd.	ASTRAL
Aurobindo Pharma Ltd.	AUROPHARMA
BSE Ltd.	BSE
Balkrishna Industries Ltd.	BALKRISIND
Bandhan Bank Ltd.	BANDHANBNK
Bank of India	BANKINDIA
Bank of Maharashtra	MAHABANK
Berger Paints India Ltd.	BERGEPAINT
Bharat Dynamics Ltd.	BDL
Bharat Forge Ltd.	BHARATFORG
Bharat Heavy Electricals Ltd.	BHEL
Bharti Hexacom Ltd.	BHARTIHEXA
Biocon Ltd.	BIOCON
Blue Star Ltd.	BLUESTARCO
CRISIL Ltd.	CRISIL
Cochin Shipyard Ltd.	COCHINSHIP
Coforge Ltd.	COFORGE
Colgate Palmolive (India) Ltd.	COLPAL
Container Corporation of India Ltd.	CONCOR
Coromandel International Ltd.	COROMANDEL
Cummins India Ltd.	CUMMINSIND
Dalmia Bharat Ltd.	DALBHARAT
Deepak Nitrite Ltd.	DEEPAKNTR
Dixon Technologies (India) Ltd.	DIXON
Emami Ltd.	EMAMILTD
Endurance Technologies Ltd.	ENDURANCE
Escorts Kubota Ltd.	ESCORTS
Exide Industries Ltd.	EXIDEIND
FSN E-Commerce Ventures Ltd.	NYKAA
Federal Bank Ltd.	FEDERALBNK
Fortis Healthcare Ltd.	FORTIS
GE Vernova T&D India Ltd.	GVT&D
GMR Airports Ltd.	GMRAIRPORT
General Insurance Corporation of India	GICRE
Gland Pharma Ltd.	GLAND
Glaxosmithkline Pharmaceuticals Ltd.	GLAXO
Glenmark Pharmaceuticals Ltd.	GLENMARK
Global Health Ltd.	MEDANTA
Godrej Industries Ltd.	GODREJIND
Godrej Properties Ltd.	GODREJPROP
Gujarat Fluorochemicals Ltd.	FLUOROCHEM
Gujarat Gas Ltd.	GUJGASLTD
HDFC Asset Management Company Ltd.	HDFCAMC
Hindustan Petroleum Corporation Ltd.	HINDPETRO
Hindustan Zinc Ltd.	HINDZINC
Hitachi Energy India Ltd.	POWERINDIA
Honeywell Automation India Ltd.	HONAUT
Housing & Urban Development Corporation Ltd.	HUDCO
IDFC First Bank Ltd.	IDFCFIRSTB
IRB Infrastructure Developers Ltd.	IRB
Indian Bank	INDIANB
Indian Railway Catering And Tourism Corporation Ltd.	IRCTC
Indian Renewable Energy Development Agency Ltd.	IREDA
Indraprastha Gas Ltd.	IGL
Indus Towers Ltd.	INDUSTOWER
Ipca Laboratories Ltd.	IPCALAB
J.K. Cement Ltd.	JKCEMENT
JSW Infrastructure Ltd.	JSWINFRA
Jindal Stainless Ltd.	JSL
Jubilant Foodworks Ltd.	JUBLFOOD
K.P.R. Mill Ltd.	KPRMILL
KEI Industries Ltd.	KEI
KPIT Technologies Ltd.	KPITTECH
Kalyan Jewellers India Ltd.	KALYANKJIL
L&T Finance Ltd.	LTF
L&T Technology Services Ltd.	LTTS
LIC Housing Finance Ltd.	LICHSGFIN
Linde India Ltd.	LINDEINDIA
Lloyds Metals And Energy Ltd.	LLOYDSME
Lupin Ltd.	LUPIN
MRF Ltd.	MRF
Mahindra & Mahindra Financial Services Ltd.	M&MFIN
Mangalore Refinery & Petrochemicals Ltd.	MRPL
Mankind Pharma Ltd.	MANKIND
Marico Ltd.	MARICO
Max Financial Services Ltd.	MFSL
Max Healthcare Institute Ltd.	MAXHEALTH
Mazagoan Dock Shipbuilders Ltd.	MAZDOCK
Motherson Sumi Wiring India Ltd.	MSUMI
Motilal Oswal Financial Services Ltd.	MOTILALOFS
MphasiS Ltd.	MPHASIS
Muthoot Finance Ltd.	MUTHOOTFIN
NHPC Ltd.	NHPC
Aadhar Housing Finance Ltd.	AADHARHFC
Aarti Industries Ltd.	AARTIIND
Aavas Financiers Ltd.	AAVAS
Action Construction Equipment Ltd.	ACE
Aditya Birla Real Estate Ltd.	ABREL
Aditya Birla Sun Life AMC Ltd.	ABSLAMC
Aegis Logistics Ltd.	AEGISLOG
Affle (India) Ltd.	AFFLE
Alembic Pharmaceuticals Ltd.	APLLTD
Alkyl Amines Chemicals Ltd.	ALKYLAMINE
Alok Industries Ltd.	ALOKINDS
Amara Raja Energy & Mobility Ltd.	ARE&M
Amber Enterprises India Ltd.	AMBER
Anand Rathi Wealth Ltd.	ANANDRATHI
Anant Raj Ltd.	ANANTRAJ
Angel One Ltd.	ANGELONE
Aptus Value Housing Finance India Ltd.	APTUS
Asahi India Glass Ltd.	ASAHIINDIA
Aster DM Healthcare Ltd.	ASTERDM
AstraZenca Pharma India Ltd.	ASTRAZEN
Atul Ltd.	ATUL
Authum Investment & Infrastructure Ltd.	AIIL
BASF India Ltd.	BASF
BEML Ltd.	BEML
BLS International Services Ltd.	BLS
Balrampur Chini Mills Ltd.	BALRAMCHIN
Bata India Ltd.	BATAINDIA
Bayer Cropscience Ltd.	BAYERCROP
Bikaji Foods International Ltd.	BIKAJI
Birlasoft Ltd.	BSOFT
Blue Dart Express Ltd.	BLUEDART
Bombay Burmah Trading Corporation Ltd.	BBTC
Brigade Enterprises Ltd.	BRIGADE
C.E. Info Systems Ltd.	MAPMYINDIA
CCL Products (I) Ltd.	CCL
CESC Ltd.	CESC
Campus Activewear Ltd.	CAMPUS
Can Fin Homes Ltd.	CANFINHOME
Caplin Point Laboratories Ltd.	CAPLIPOINT
Capri Global Capital Ltd.	CGCL
Carborundum Universal Ltd.	CARBORUNIV
Castrol India Ltd.	CASTROLIND
Ceat Ltd.	CEATLTD
Central Bank of India	CENTRALBK
Central Depository Services (India) Ltd.	CDSL
Century Plyboards (India) Ltd.	CENTURYPLY
Cera Sanitaryware Ltd	CERA
Chalet Hotels Ltd.	CHALET
Chambal Fertilizers & Chemicals Ltd.	CHAMBLFERT
Chennai Petroleum Corporation Ltd.	CHENNPETRO
Cholamandalam Financial Holdings Ltd.	CHOLAHLDNG
City Union Bank Ltd.	CUB
Clean Science and Technology Ltd.	CLEAN
Computer Age Management Services Ltd.	CAMS
Concord Biotech Ltd.	CONCORDBIO
Craftsman Automation Ltd.	CRAFTSMAN
CreditAccess Grameen Ltd.	CREDITACC
Crompton Greaves Consumer Electricals Ltd.	CROMPTON
Cyient Ltd.	CYIENT
DCM Shriram Ltd.	DCMSHRIRAM
DOMS Industries Ltd.	DOMS
Data Patterns (India) Ltd.	DATAPATTNS
Deepak Fertilisers & Petrochemicals Corp. Ltd.	DEEPAKFERT
Delhivery Ltd.	DELHIVERY
Devyani International Ltd.	DEVYANI
Dr. Lal Path Labs Ltd.	LALPATHLAB
E.I.D. Parry (India) Ltd.	EIDPARRY
EIH Ltd.	EIHOTEL
Elecon Engineering Co. Ltd.	ELECON
Elgi Equipments Ltd.	ELGIEQUIP
Engineers India Ltd.	ENGINERSIN
Eris Lifesciences Ltd.	ERIS
Fertilisers and Chemicals Travancore Ltd.	FACT
Finolex Cables Ltd.	FINCABLES
Finolex Industries Ltd.	FINPIPE
Firstsource Solutions Ltd.	FSL
Five-Star Business Finance Ltd.	FIVESTAR
Garden Reach Shipbuilders & Engineers Ltd.	GRSE
Gillette India Ltd.	GILLETTE
Go Digit General Insurance Ltd.	GODIGIT
Godawari Power & Ispat Ltd.	GPIL
Godfrey Phillips India Ltd.	GODFRYPHLP
Godrej Agrovet Ltd.	GODREJAGRO
Granules India Ltd.	GRANULES
Graphite India Ltd.	GRAPHITE
Gravita India Ltd.	GRAVITA
Great Eastern Shipping Co. Ltd.	GESHIP
Gujarat Mineral Development Corporation Ltd.	GMDCLTD
Gujarat Narmada Valley Fertilizers and Chemicals Ltd.	GNFC
Gujarat Pipavav Port Ltd.	GPPL
Gujarat State Petronet Ltd.	GSPL
H.E.G. Ltd.	HEG
HFCL Ltd.	HFCL
Happiest Minds Technologies Ltd.	HAPPSTMNDS
Himadri Speciality Chemical Ltd.	HSCL
Hindustan Copper Ltd.	HINDCOPPER
Home First Finance Company India Ltd.	HOMEFIRST
Honasa Consumer Ltd.	HONASA
IDBI Bank Ltd.	IDBI
IFCI Ltd.	IFCI
IIFL Finance Ltd.	IIFL
INOX India Ltd.	INOXINDIA
IRCON International Ltd.	IRCON
ITI Ltd.	ITI
Indegene Ltd.	INDGN
India Cements Ltd.	INDIACEM
Indiamart Intermesh Ltd.	INDIAMART
Indian Energy Exchange Ltd.	IEX
Indian Overseas Bank	IOB
Inox Wind Ltd.	INOXWIND
Intellect Design Arena Ltd.	INTELLECT
J.B. Chemicals & Pharmaceuticals Ltd.	JBCHEPHARM
JBM Auto Ltd.	JBMA
JK Tyre & Industries Ltd.	JKTYRE
JM Financial Ltd.	JMFINANCIL
JSW Holdings Ltd.	JSWHL
Jaiprakash Power Ventures Ltd.	JPPOWER
Jammu & Kashmir Bank Ltd.	J&KBANK
Jindal Saw Ltd.	JINDALSAW
Jubilant Ingrevia Ltd.	JUBLINGREA
Jubilant Pharmova Ltd.	JUBLPHARMA
Jupiter Wagons Ltd.	JWL
Justdial Ltd.	JUSTDIAL
Jyothy Labs Ltd.	JYOTHYLAB
Jyoti CNC Automation Ltd.	JYOTICNC
KNR Constructions Ltd.	KNRCON
Kajaria Ceramics Ltd.	KAJARIACER
Kalpataru Projects International Ltd.	KPIL
Kansai Nerolac Paints Ltd.	KANSAINER
Karur Vysya Bank Ltd.	KARURVYSYA
Kaynes Technology India Ltd.	KAYNES
Kec International Ltd.	KEC
Kfin Technologies Ltd.	KFINTECH
Kirloskar Brothers Ltd.	KIRLOSBROS
Kirloskar Oil Eng Ltd.	KIRLOSENG
Krishna Institute of Medical Sciences Ltd.	KIMS
LT Foods Ltd.	LTFOODS
Latent View Analytics Ltd.	LATENTVIEW
Laurus Labs Ltd.	LAURUSLABS
Lemon Tree Hotels Ltd.	LEMONTREE
MMTC Ltd.	MMTC
Mahanagar Gas Ltd.	MGL
Maharashtra Seamless Ltd.	MAHSEAMLES
Manappuram Finance Ltd.	MANAPPURAM
Mastek Ltd.	MASTEK
Metropolis Healthcare Ltd.	METROPOLIS
Minda Corporation Ltd.	MINDACORP
Multi Commodity Exchange of India Ltd.	MCX
NATCO Pharma Ltd.	NATCOPHARM
NBCC (India) Ltd.	NBCC
NCC Ltd.	NCC
NMDC Steel Ltd.	NSLNISP
Narayana Hrudayalaya Ltd.	NH
Nava Ltd.	NAVA
Navin Fluorine International Ltd.	NAVINFLUOR
Netweb Technologies India Ltd.	NETWEB
Network18 Media & Investments Ltd.	NETWORK18
Neuland Laboratories Ltd.	NEULANDLAB
Newgen Software Technologies Ltd.	NEWGEN
Nuvama Wealth Management Ltd.	NUVAMA
Olectra Greentech Ltd.	OLECTRA
PCBL Chemical Ltd.	PCBL
PG Electroplast Ltd.	PGEL
PNB Housing Finance Ltd.	PNBHOUSING
PNC Infratech Ltd.	PNCINFRA
PTC Industries Ltd.	PTCIL
PVR INOX Ltd.	PVRINOX
Pfizer Ltd.	PFIZER
Piramal Enterprises Ltd.	PEL
Piramal Pharma Ltd.	PPLPHARMA
Poly Medicure Ltd.	POLYMED
Poonawalla Fincorp Ltd.	POONAWALLA
Praj Industries Ltd.	PRAJIND
Quess Corp Ltd.	QUESS
R R Kabel Ltd.	RRKABEL
RBL Bank Ltd.	RBLBANK
RHI MAGNESITA INDIA LTD.	RHIM
RITES Ltd.	RITES
Radico Khaitan Ltd	RADICO
Railtel Corporation Of India Ltd.	RAILTEL
Rainbow Childrens Medicare Ltd.	RAINBOW
Ramkrishna Forgings Ltd.	RKFORGE
Rashtriya Chemicals & Fertilizers Ltd.	RCF
RattanIndia Enterprises Ltd.	RTNINDIA
Raymond Ltd.	RAYMOND
Redington Ltd.	REDINGTON
Reliance Power Ltd.	RPOWER
Route Mobile Ltd.	ROUTE
SBFC Finance Ltd.	SBFC
SKF India Ltd.	SKFINDIA
Sammaan Capital Ltd.	SAMMAANCAP
Sapphire Foods India Ltd.	SAPPHIRE
Sarda Energy and Minerals Ltd.	SARDAEN
Saregama India Ltd	SAREGAMA
Schneider Electric Infrastructure Ltd.	SCHNEIDER
Shipping Corporation of India Ltd.	SCI
Shree Renuka Sugars Ltd.	RENUKA
Shyam Metalics and Energy Ltd.	SHYAMMETL
Signatureglobal (India) Ltd.	SIGNATURE
Sobha Ltd.	SOBHA
Sonata Software Ltd.	SONATSOFTW
Sterling and Wilson Renewable Energy Ltd.	SWSOLAR
Sumitomo Chemical India Ltd.	SUMICHEM
Suven Pharmaceuticals Ltd.	SUVENPHAR
Swan Energy Ltd.	SWANENERGY
Syrma SGS Technology Ltd.	SYRMA
TBO Tek Ltd.	TBOTEK
Tanla Platforms Ltd.	TANLA
Tata Chemicals Ltd.	TATACHEM
Tata Teleservices (Maharashtra) Ltd.	TTML
Techno Electric & Engineering Company Ltd.	TECHNOE
Tejas Networks Ltd.	TEJASNET
The Ramco Cements Ltd.	RAMCOCEM
Timken India Ltd.	TIMKEN
Titagarh Rail Systems Ltd.	TITAGARH
Transformers And Rectifiers (India) Ltd.	TARIL
Trident Ltd.	TRIDENT
Triveni Engineering & Industries Ltd.	TRIVENI
Triveni Turbine Ltd.	TRITURBINE
UCO Bank	UCOBANK
UTI Asset Management Company Ltd.	UTIAMC
Usha Martin Ltd.	USHAMART
V-Guard Industries Ltd.	VGUARD
Valor Estate Ltd.	DBREALTY
Vardhman Textiles Ltd.	VTL
Vedant Fashions Ltd.	MANYAVAR
Vijaya Diagnostic Centre Ltd.	VIJAYA
Welspun Corp Ltd.	WELCORP
Welspun Living Ltd.	WELSPUNLIV
Westlife Foodworld Ltd.	WESTLIFE
Whirlpool of India Ltd.	WHIRLPOOL
Wockhardt Ltd.	WOCKPHARMA
ZF Commercial Vehicle Control Systems India Ltd.	ZFCVINDIA
Zee Entertainment Enterprises Ltd.	ZEEL
Zen Technologies Ltd.	ZENTEC
Zensar Technolgies Ltd.	ZENSARTECH
eClerx Services Ltd.	ECLERX
AGI Greenpac Ltd.	AGI
ASK Automotive Ltd.	ASKAUTOLTD
Aarti Drugs Ltd.	AARTIDRUGS
Aarti Pharmalabs Ltd.	AARTIPHARM
Advanced Enzyme Tech Ltd.	ADVENZYMES
Aether Industries Ltd.	AETHER
Ahluwalia Contracts (India) Ltd.	AHLUCONT
Akzo Nobel India Ltd.	AKZOINDIA
Allcargo Logistics Ltd.	ALLCARGO
Ami Organics Ltd.	AMIORG
Apeejay Surrendra Park Hotels Ltd.	PARKHOTELS
Archean Chemical Industries Ltd.	ACI
Arvind Fashions Ltd.	ARVINDFASN
Arvind Ltd.	ARVIND
Ashoka Buildcon Ltd.	ASHOKA
Astra Microwave Products Ltd.	ASTRAMICRO
Aurionpro Solution Ltd.	AURIONPRO
Avalon Technologies Ltd.	AVALON
Avanti Feeds Ltd.	AVANTIFEED
Awfis Space Solutions Ltd.	AWFIS
Azad Engineering Ltd.	AZAD
Bajaj Hindusthan Sugar Ltd.	BAJAJHIND
Balaji Amines Ltd.	BALAMINES
Balu Forge Industries Ltd.	BALUFORGE
Banco Products (India) Ltd.	BANCOINDIA
Bhansali Engineering Polymers Ltd.	BEPL
Bharat Bijlee Ltd.	BBL
Birla Corporation Ltd.	BIRLACORPN
Blue Jet Healthcare Ltd.	BLUEJET
Bombay Dyeing & Manufacturing Co. Ltd.	BOMDYEING
Borosil Ltd.	BOROLTD
Borosil Renewables Ltd.	BORORENEW
CIE Automotive India Ltd.	CIEINDIA
CMS Info Systems Ltd.	CMSINFO
CSB Bank Ltd.	CSBBANK
Cartrade Tech Ltd.	CARTRADE"""

stocks_df1 = parse_stock_list(stock_list_text1)
print('len(stocks_df1)', len(stocks_df1))

len(stocks_df1) 422


In [107]:
stock_list_text2  = """Company Name	Symbol
Cello World Ltd.	CELLO
Chemplast Sanmar Ltd.	CHEMPLASTS
Choice International Ltd.	CHOICEIN
Cigniti Technologies Ltd.	CIGNITITEC
Cyient DLM Ltd.	CYIENTDLM
DCB Bank Ltd.	DCBBANK
DCX Systems Ltd.	DCXINDIA
Datamatics Global Services Ltd.	DATAMATICS
Dhani Services Ltd.	DHANI
Dilip Buildcon Ltd.	DBL
Dishman Carbogen Amcis Ltd.	DCAL
Dodla Dairy Ltd.	DODLA
Dynamatic Technologies Ltd.	DYNAMATECH
EPL Ltd.	EPL
Easy Trip Planners Ltd.	EASEMYTRIP
Edelweiss Financial Services Ltd.	EDELWEISS
Electronics Mart India Ltd.	EMIL
Electrosteel Castings Ltd.	ELECTCAST
Entero Healthcare Solutions Ltd.	ENTERO
Epigral Ltd.	EPIGRAL
Equitas Small Finance Bank Ltd.	EQUITASBNK
Ethos Ltd.	ETHOSLTD
FDC Ltd.	FDC
Fiem Industries Ltd	FIEMIND
Fine Organic Industries Ltd.	FINEORG
Fineotex Chemical Ltd.	FCL
Force Motors Ltd.	FORCEMOT
G R Infraprojects Ltd.	GRINFRA
GHCL Ltd.	GHCL
GMM Pfaudler Ltd.	GMMPFAUDLR
GMR Power and Urban Infra Ltd.	GMRP&UI
Gabriel India Ltd.	GABRIEL
Ganesh Housing Corporation Ltd.	GANESHHOUC
Ganesha Ecosphere Ltd.	GANECOS
Garware Hi-Tech Films Ltd.	GRWRHITECH
Garware Technical Fibres Ltd.	GARFIBRES
Gateway Distriparks Ltd.	GATEWAY
Gokaldas Exports Ltd.	GOKEX
Gopal Snacks Ltd.	GOPAL
Greaves Cotton Ltd.	GREAVESCOT
Greenpanel Industries Ltd.	GREENPANEL
Greenply Industries Ltd.	GREENPLY
Gujarat Ambuja Exports Ltd.	GAEL
Gujarat State Fertilizers & Chemicals Ltd.	GSFC
Gulf Oil Lubricants India Ltd.	GULFOILLUB
H.G. Infra Engineering Ltd.	HGINFRA
Hathway Cable & Datacom Ltd.	HATHWAY
Healthcare Global Enterprises Ltd.	HCG
HeidelbergCement India Ltd.	HEIDELBERG
Hemisphere Properties India Ltd.	HEMIPROP
Heritage Foods Ltd.	HERITGFOOD
Hikal Ltd.	HIKAL
Hindustan Construction Co. Ltd.	HCC
IFB Industries Ltd.	IFBIND
ITD Cementation India Ltd.	ITDCEM
Imagicaaworld Entertainment Ltd.	IMAGICAA
India Glycols Ltd.	INDIAGLYCO
India Shelter Finance Corporation Ltd.	INDIASHLTR
Indian Metals & Ferro Alloys Ltd.	IMFA
Indigo Paints Ltd.	INDIGOPNTS
Indo Count Industries Ltd.	ICIL
Infibeam Avenues Ltd.	INFIBEAM
Ingersoll Rand (India) Ltd.	INGERRAND
Innova Captab Ltd.	INNOVACAP
Inox Green Energy Services Ltd.	INOXGREEN
Ion Exchange (India) Ltd.	IONEXCHANG
Isgec Heavy Engineering Ltd.	ISGEC
J.Kumar Infraprojects Ltd.	JKIL
JK Lakshmi Cement Ltd.	JKLAKSHMI
JK Paper Ltd.	JKPAPER
JTL Industries Ltd.	JTLIND
Jai Balaji Industries Ltd.	JAIBALAJI
Jai Corp Ltd.	JAICORPLTD
Jain Irrigation Systems Ltd.	JISLJALEQS
Jamna Auto Industries Ltd.	JAMNAAUTO
Jana Small Finance Bank Ltd.	JSFB
Jindal Worldwide Ltd.	JINDWORLD
Johnson Controls - Hitachi Air Conditioning India Ltd.	JCHAC
KPI Green Energy Ltd.	KPIGREEN
KRBL Ltd.	KRBL
KSB Ltd.	KSB
Kalyani Steels Ltd.	KSL
Karnataka Bank Ltd.	KTKBANK
Kaveri Seed Company Ltd.	KSCL
Kirloskar Pneumatic Company Ltd.	KIRLPNU
LMW Ltd.	LMW
Laxmi Organic Industries Ltd.	LXCHEM
Le Travenues Technology Ltd.	IXIGO
Lloyds Engineering Works Ltd.	LLOYDSENGG
Lux Industries Ltd.	LUXIND
MOIL Ltd.	MOIL
MSTC Ltd.	MSTCLTD
MTAR Technologies Ltd.	MTARTECH
Maharashtra Scooters Ltd.	MAHSCOOTER
Mahindra Lifespace Developers Ltd.	MAHLIFE
Man Infraconstruction Ltd.	MANINFRA
Marksans Pharma Ltd.	MARKSANS
Max Estates Ltd.	MAXESTATES
Medplus Health Services Ltd.	MEDPLUS
Mishra Dhatu Nigam Ltd.	MIDHANI
Mrs. Bectors Food Specialities Ltd.	BECTORFOOD
NEOGEN CHEMICALS LTD.	NEOGEN
NESCO Ltd.	NESCO
NOCIL Ltd.	NOCIL
National Fertilizers Ltd.	NFL
Nazara Technologies Ltd.	NAZARA
Nuvoco Vistas Corporation Ltd.	NUVOCO
Optiemus Infracom Ltd.	OPTIEMUS
Orchid Pharma Ltd.	ORCHPHARMA
Orient Cement Ltd.	ORIENTCEM
Orissa Min Dev Co Ltd.	ORISSAMINE
PC Jeweller Ltd.	PCJEWELLER
PTC India Ltd.	PTC
Paisalo Digital Ltd.	PAISALO
Paradeep Phosphates Ltd.	PARADEEP
Paras Defence and Space Technologies Ltd.	PARAS
Patel Engineering Ltd.	PATELENG
Pearl Global Industries Ltd.	PGIL
Polyplex Corporation Ltd.	POLYPLEX
Power Mech Projects Ltd.	POWERMECH
Pricol Ltd.	PRICOLLTD
Prince Pipes and Fittings Ltd.	PRINCEPIPE
Prism Johnson Ltd.	PRSMJOHNSN
Prudent Corporate Advisory Services Ltd.	PRUDENT
Rain Industries Ltd	RAIN
Rajesh Exports Ltd.	RAJESHEXPO
Rallis India Ltd.	RALLIS
Rategain Travel Technologies Ltd.	RATEGAIN
RattanIndia Power Ltd.	RTNPOWER
Redtape Ltd.	REDTAPE
Refex Industries Ltd.	REFEX
Reliance Infrastructure Ltd.	RELINFRA
Religare Enterprises Ltd.	RELIGARE
Responsive Industries Ltd.	RESPONIND
Restaurant Brands Asia Ltd.	RBA
Rossari Biotech Ltd.	ROSSARI
Safari Industries (India) Ltd.	SAFARI
Samhi Hotels Ltd.	SAMHI
Sanofi India Ltd.	SANOFI
Sansera Engineering Ltd.	SANSERA
Senco Gold Ltd.	SENCO
Sequent Scientific Ltd.	SEQUENT
Shaily Engineering Plastics Ltd.	SHAILY
Shakti Pumps (India) Ltd.	SHAKTIPUMP
Sharda Cropchem Ltd.	SHARDACROP
Share India Securities Ltd.	SHAREINDIA
Sheela Foam Ltd.	SFL
Shilpa Medicare Ltd.	SHILPAMED
Shivalik Bimetal Controls Ltd.	SBCL
Shoppers Stop Ltd.	SHOPERSTOP
Shriram Pistons & Rings Ltd.	SHRIPISTON
Skipper Ltd.	SKIPPER
South Indian Bank Ltd.	SOUTHBANK
Spandana Sphoorty Financial Ltd.	SPANDANA
Star Cement Ltd.	STARCEMENT
Sterlite Technologies Ltd.	STLTECH
Strides Pharma Science Ltd.	STAR
Stylam Industries Ltd.	STYLAMIND
Subros Ltd.	SUBROS
Sudarshan Chemical Industries Ltd.	SUDARSCHEM
Sula Vineyards Ltd.	SULA
Sun Pharma Advanced Research Company Ltd.	SPARC
Sunflag Iron & Steel Company Ltd.	SUNFLAG
Sunteck Realty Ltd.	SUNTECK
Suprajit Engineering Ltd.	SUPRAJIT
Supriya Lifescience Ltd.	SUPRIYA
Surya Roshni Ltd.	SURYAROSNI
Symphony Ltd.	SYMPHONY
TARC Ltd.	TARC
TD Power Systems Ltd.	TDPOWERSYS
TVS Supply Chain Solutions Ltd.	TVSSCS
Teamlease Services Ltd.	TEAMLEASE
Technocraft Industries (India) Ltd.	TIIL
Tega Industries Ltd.	TEGA
Texmaco Rail & Eng. Ltd.	TEXRAIL
Thangamayil Jewellery Ltd.	THANGAMAYL
The Anup Engineering Ltd.	ANUP
Thirumalai Chemicals Ltd.	TIRUMALCHM
Thomas Cook (India) Ltd.	THOMASCOOK
Tilaknagar Industries Ltd.	TI
Time Technoplast Ltd.	TIMETECHNO
Tips Music Ltd.	TIPSMUSIC
Ujjivan Small Finance Bank Ltd.	UJJIVANSFB
V-Mart Retail Ltd.	VMART
V.I.P. Industries Ltd.	VIPIND
VST Industries Ltd.	VSTIND
Va Tech Wabag Ltd.	WABAG
Vaibhav Global Ltd.	VAIBHAVGBL
Varroc Engineering Ltd.	VARROC
Venus Pipes & Tubes Ltd.	VENUSPIPES
Vesuvius India Ltd.	VESUVIUS
Voltamp Transformers Ltd	VOLTAMP
Websol Energy System Ltd.	WEBELSOLAR
Welspun Enterprises Ltd.	WELENT
Wonderla Holidays Ltd.	WONDERLA
Yatharth Hospital & Trauma Care Services Ltd.	YATHARTH
Zaggle Prepaid Ocean Services Ltd.	ZAGGLE
Zydus Wellness Ltd.	ZYDUSWELL
eMudhra Ltd.	EMUDHRA"""

stocks_df2 = parse_stock_list(stock_list_text2)
print('len(stocks_df2)', len(stocks_df2))

len(stocks_df2) 199


In [ ]:
analyze_stocks(stocks_df1)

2025-04-12 22:47:08,894 - INFO - Processing: Rail Vikas Nigam Ltd. (RVNL)
2025-04-12 22:47:08,897 - INFO - Processing: Adani Enterprises Ltd. (ADANIENT)
2025-04-12 22:47:08,899 - INFO - Processing: Adani Ports and Special Economic Zone Ltd. (ADANIPORTS)
2025-04-12 22:47:08,897 - INFO - Processing: Adani Enterprises Ltd. (ADANIENT)
2025-04-12 22:47:08,899 - INFO - Processing: Adani Ports and Special Economic Zone Ltd. (ADANIPORTS)
2025-04-12 22:47:08,902 - INFO - Processing: Apollo Hospitals Enterprise Ltd. (APOLLOHOSP)
2025-04-12 22:47:08,904 - INFO - Processing: Asian Paints Ltd. (ASIANPAINT)
2025-04-12 22:47:08,902 - INFO - Processing: Apollo Hospitals Enterprise Ltd. (APOLLOHOSP)
2025-04-12 22:47:08,904 - INFO - Processing: Asian Paints Ltd. (ASIANPAINT)
2025-04-12 22:47:12,055 - INFO - Verified unique data for RVNL: Latest price = ₹346.30
2025-04-12 22:47:12,070 - INFO - Verified price data for RVNL: Current price = ₹346.30
2025-04-12 22:47:12,055 - INFO - Verified unique data for 

In [ ]:
analyze_stocks(stocks_df2)